# 2. RAG (Retrieval-Augmented Generation) Application
## Build Production-Ready RAG with Cohere Embeddings + AWS Bedrock

This notebook demonstrates:
- Document ingestion and chunking strategies
- Vector store integration with FAISS
- Retrieval-augmented generation using Claude on Bedrock
- Context-aware question answering
- Citation and source tracking
- Evaluation metrics for RAG quality

## Setup and Installation

In [1]:
# Install required libraries
!pip install boto3 faiss-cpu numpy pandas tiktoken langchain-text-splitters -q

print("✅ Installation complete")

✅ Installation complete


In [2]:
import boto3
import json
import numpy as np
import faiss
import re
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## AWS Bedrock Configuration

In [3]:
# AWS Configuration
AWS_REGION = 'us-east-1'
EMBEDDING_MODEL_ID = 'cohere.embed-v4:0'
LLM_MODEL_ID = 'anthropic.claude-3-sonnet-20240229-v1:0'  # or claude-3-5-sonnet

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION
)

print(f"✅ AWS Bedrock configured")
print(f"📦 Embedding Model: {EMBEDDING_MODEL_ID}")
print(f"🤖 LLM Model: {LLM_MODEL_ID}")

✅ AWS Bedrock configured
📦 Embedding Model: cohere.embed-v4:0
🤖 LLM Model: anthropic.claude-3-sonnet-20240229-v1:0


## Document Data Classes

In [4]:
@dataclass
class Document:
    """Represents a document chunk"""
    content: str
    metadata: Dict
    doc_id: str = ""
    
    def __post_init__(self):
        if not self.doc_id:
            self.doc_id = f"doc_{hash(self.content) % 10**8}"


@dataclass
class RetrievalResult:
    """Represents a retrieval result"""
    document: Document
    score: float
    rank: int


@dataclass
class RAGResponse:
    """Represents a complete RAG response"""
    query: str
    answer: str
    sources: List[RetrievalResult]
    context_used: str
    timestamp: str

print("✅ Data classes defined")

✅ Data classes defined


## Text Chunking Strategies

In [5]:
class TextChunker:
    """Advanced text chunking with multiple strategies"""
    
    @staticmethod
    def chunk_by_tokens(
        text: str,
        chunk_size: int = 512,
        overlap: int = 50
    ) -> List[str]:
        """
        Chunk text by approximate token count
        
        Args:
            text: Input text
            chunk_size: Target chunk size in tokens
            overlap: Overlap between chunks
        
        Returns:
            List of text chunks
        """
        # Approximate: 1 token ≈ 4 characters
        char_chunk_size = chunk_size * 4
        char_overlap = overlap * 4
        
        chunks = []
        start = 0
        
        while start < len(text):
            end = start + char_chunk_size
            
            # Try to break at sentence boundary
            if end < len(text):
                # Look for sentence ending
                for punct in ['. ', '! ', '? ', '\n\n']:
                    last_punct = text[start:end].rfind(punct)
                    if last_punct != -1:
                        end = start + last_punct + len(punct)
                        break
            
            chunk = text[start:end].strip()
            if chunk:
                chunks.append(chunk)
            
            start = end - char_overlap
        
        return chunks
    
    @staticmethod
    def chunk_by_paragraphs(
        text: str,
        min_chunk_size: int = 100,
        max_chunk_size: int = 2000
    ) -> List[str]:
        """
        Chunk text by paragraphs
        
        Args:
            text: Input text
            min_chunk_size: Minimum characters per chunk
            max_chunk_size: Maximum characters per chunk
        
        Returns:
            List of text chunks
        """
        paragraphs = re.split(r'\n\n+', text)
        chunks = []
        current_chunk = []
        current_size = 0
        
        for para in paragraphs:
            para = para.strip()
            if not para:
                continue
            
            para_size = len(para)
            
            if current_size + para_size > max_chunk_size and current_chunk:
                chunks.append('\n\n'.join(current_chunk))
                current_chunk = [para]
                current_size = para_size
            else:
                current_chunk.append(para)
                current_size += para_size
        
        if current_chunk:
            chunks.append('\n\n'.join(current_chunk))
        
        return chunks
    
    @staticmethod
    def chunk_by_sentences(
        text: str,
        sentences_per_chunk: int = 5,
        overlap_sentences: int = 1
    ) -> List[str]:
        """
        Chunk text by sentences
        
        Args:
            text: Input text
            sentences_per_chunk: Number of sentences per chunk
            overlap_sentences: Overlap in sentences
        
        Returns:
            List of text chunks
        """
        # Simple sentence splitting
        sentences = re.split(r'(?<=[.!?])\s+', text)
        chunks = []
        
        for i in range(0, len(sentences), sentences_per_chunk - overlap_sentences):
            chunk_sentences = sentences[i:i + sentences_per_chunk]
            if chunk_sentences:
                chunks.append(' '.join(chunk_sentences))
        
        return chunks

print("✅ TextChunker class defined")

✅ TextChunker class defined


## Embedding and Vector Store

In [16]:
def get_embeddings(texts: List[str], input_type: str = "search_document") -> np.ndarray:
    """Generate embeddings using Cohere on Bedrock"""
    body = json.dumps({
        "texts": texts,
        "input_type": input_type,
        "embedding_types": ["float"],
        "truncate": "END"
    })
    
    response = bedrock_runtime.invoke_model(
        modelId=EMBEDDING_MODEL_ID,
        body=body
    )
    
    result = json.loads(response['body'].read())
    return np.array(result['embeddings']['float'], dtype=np.float32)


class RAGVectorStore:
    """Vector store optimized for RAG applications"""
    
    def __init__(self):
        """Initialize without dimension - auto-detect from first batch"""
        self.dimension = None
        self.index = None
        self.documents: List[Document] = []
    
    def add_documents(self, documents: List[Document], batch_size: int = 50):
        """Add documents to the vector store"""
        contents = [doc.content for doc in documents]
        
        all_embeddings = []
        for i in range(0, len(contents), batch_size):
            batch = contents[i:i + batch_size]
            embeddings = get_embeddings(batch, input_type="search_document")
            
            # Auto-detect dimension from first batch
            if self.index is None:
                self.dimension = embeddings.shape[1]
                self.index = faiss.IndexFlatIP(self.dimension)
                print(f"✅ Auto-detected dimension: {self.dimension}")
            
            faiss.normalize_L2(embeddings)
            all_embeddings.append(embeddings)
        
        if all_embeddings:
            all_embeddings = np.vstack(all_embeddings)
            self.index.add(all_embeddings)
            self.documents.extend(documents)
        
        print(f"✅ Added {len(documents)} documents")
    
    def search(
        self,
        query: str,
        top_k: int = 5,
        min_score: float = 0.0
    ) -> List[RetrievalResult]:
        """Search for relevant documents"""
        # Generate query embedding
        query_embedding = get_embeddings([query], input_type="search_query")
        faiss.normalize_L2(query_embedding)
        
        # Search
        scores, indices = self.index.search(query_embedding, top_k)
        
        # Prepare results
        results = []
        for rank, (score, idx) in enumerate(zip(scores[0], indices[0]), 1):
            if score >= min_score and idx < len(self.documents):
                results.append(RetrievalResult(
                    document=self.documents[idx],
                    score=float(score),
                    rank=rank
                ))
        
        return results

print("✅ RAGVectorStore class defined")

✅ RAGVectorStore class defined


## LLM Generation with Claude

In [17]:
def generate_with_claude(
    prompt: str,
    max_tokens: int = 2000,
    temperature: float = 0.7
) -> str:
    """
    Generate response using Claude on Bedrock
    
    Args:
        prompt: Input prompt
        max_tokens: Maximum tokens to generate
        temperature: Sampling temperature
    
    Returns:
        Generated text
    """
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    })
    
    response = bedrock_runtime.invoke_model(
        modelId=LLM_MODEL_ID,
        body=body
    )
    
    result = json.loads(response['body'].read())
    return result['content'][0]['text']

print("✅ Claude generation function defined")

✅ Claude generation function defined


## RAG System Implementation

In [18]:
class RAGSystem:
    """Complete RAG system with retrieval and generation"""
    
    def __init__(self, vector_store: RAGVectorStore):
        self.vector_store = vector_store
        self.conversation_history = []
    
    def build_rag_prompt(
        self,
        query: str,
        context_documents: List[RetrievalResult],
        include_sources: bool = True
    ) -> str:
        """
        Build RAG prompt with retrieved context
        
        Args:
            query: User query
            context_documents: Retrieved documents
            include_sources: Whether to include source references
        
        Returns:
            Formatted prompt
        """
        # Build context section
        context_parts = []
        for i, result in enumerate(context_documents, 1):
            source_info = ""
            if include_sources and result.document.metadata:
                metadata_str = ", ".join(
                    f"{k}: {v}" for k, v in result.document.metadata.items()
                )
                source_info = f" [{metadata_str}]"
            
            context_parts.append(
                f"[Document {i}]{source_info}\n{result.document.content}"
            )
        
        context = "\n\n".join(context_parts)
        
        # Build complete prompt
        prompt = f"""You are a helpful AI assistant. Answer the user's question based on the provided context documents.

Context Documents:
{context}

User Question: {query}

Instructions:
1. Answer the question using ONLY information from the provided context
2. If the context doesn't contain enough information, say so clearly
3. Cite which document(s) you used by referencing [Document X]
4. Be concise but comprehensive
5. If multiple documents contain relevant information, synthesize them

Answer:"""
        
        return prompt
    
    def query(
        self,
        question: str,
        top_k: int = 3,
        min_score: float = 0.3,
        temperature: float = 0.7
    ) -> RAGResponse:
        """
        Process a query through the RAG pipeline
        
        Args:
            question: User question
            top_k: Number of documents to retrieve
            min_score: Minimum relevance score
            temperature: LLM temperature
        
        Returns:
            RAG response with answer and sources
        """
        # Step 1: Retrieve relevant documents
        print(f"🔍 Retrieving top {top_k} relevant documents...")
        retrieved_docs = self.vector_store.search(
            question,
            top_k=top_k,
            min_score=min_score
        )
        
        if not retrieved_docs:
            return RAGResponse(
                query=question,
                answer="I couldn't find relevant information to answer your question.",
                sources=[],
                context_used="",
                timestamp=datetime.now().isoformat()
            )
        
        print(f"✅ Retrieved {len(retrieved_docs)} documents")
        
        # Step 2: Build prompt with context
        prompt = self.build_rag_prompt(question, retrieved_docs)
        
        # Step 3: Generate answer
        print("🤖 Generating answer...")
        answer = generate_with_claude(prompt, temperature=temperature)
        
        # Step 4: Create response
        response = RAGResponse(
            query=question,
            answer=answer,
            sources=retrieved_docs,
            context_used=prompt,
            timestamp=datetime.now().isoformat()
        )
        
        # Store in history
        self.conversation_history.append(response)
        
        return response
    
    def display_response(self, response: RAGResponse):
        """Display formatted RAG response"""
        print("\n" + "="*80)
        print(f"QUESTION: {response.query}")
        print("="*80)
        print(f"\nANSWER:\n{response.answer}")
        print("\n" + "-"*80)
        print(f"SOURCES ({len(response.sources)} documents):")
        print("-"*80)
        
        for source in response.sources:
            print(f"\n[{source.rank}] Score: {source.score:.4f}")
            if source.document.metadata:
                print(f"    Metadata: {source.document.metadata}")
            print(f"    Content: {source.document.content[:150]}...")
        
        print("\n" + "="*80)

print("✅ RAGSystem class defined")

✅ RAGSystem class defined


## Sample Knowledge Base: AWS Documentation

In [19]:
# Sample AWS documentation content
aws_documentation = """
Amazon Bedrock Overview

Amazon Bedrock is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI.

With Amazon Bedrock, you can easily experiment with and evaluate top foundation models for your use case, privately customize them with your data using techniques such as fine-tuning and Retrieval Augmented Generation (RAG), and build agents that execute tasks using your enterprise systems and data sources.

Amazon SageMaker Features

Amazon SageMaker is a fully managed machine learning service. With SageMaker, data scientists and developers can quickly and easily build and train machine learning models, and then directly deploy them into a production-ready hosted environment.

SageMaker provides an integrated Jupyter authoring notebook instance for easy access to your data sources for exploration and analysis, so you don't have to manage servers. It also provides common machine learning algorithms that are optimized to run efficiently against extremely large data in a distributed environment.

AWS Lambda Functions

AWS Lambda is a serverless compute service that lets you run code without provisioning or managing servers, creating workload-aware cluster scaling logic, maintaining event integrations, or managing runtimes.

With Lambda, you can run code for virtually any type of application or backend service - all with zero administration. Just upload your code as a ZIP file or container image, and Lambda automatically and precisely allocates compute execution power and runs your code based on the incoming request or event, for any scale of traffic.

Amazon S3 Storage

Amazon Simple Storage Service (Amazon S3) is an object storage service that offers industry-leading scalability, data availability, security, and performance. Customers of all sizes and industries can use Amazon S3 to store and protect any amount of data for a range of use cases.

Amazon S3 provides management features so that you can optimize, organize, and configure access to your data to meet your specific business, organizational, and compliance requirements.

Amazon EC2 Instances

Amazon Elastic Compute Cloud (Amazon EC2) provides scalable computing capacity in the Amazon Web Services (AWS) Cloud. Using Amazon EC2 eliminates your need to invest in hardware up front, so you can develop and deploy applications faster.

You can use Amazon EC2 to launch as many or as few virtual servers as you need, configure security and networking, and manage storage. Amazon EC2 enables you to scale up or down to handle changes in requirements or spikes in popularity, reducing your need to forecast traffic.

Amazon VPC Networking

Amazon Virtual Private Cloud (Amazon VPC) lets you provision a logically isolated section of the AWS Cloud where you can launch AWS resources in a virtual network that you define. You have complete control over your virtual networking environment.

With Amazon VPC, you can select your own IP address range, create subnets, and configure route tables and network gateways. You can use both IPv4 and IPv6 for most resources in your VPC, helping to ensure secure and easy access to resources and applications.
"""

print(f"✅ Loaded AWS documentation ({len(aws_documentation)} characters)")

✅ Loaded AWS documentation (3405 characters)


## Build RAG Knowledge Base

In [20]:
print("=" * 80)
print("Building RAG Knowledge Base")
print("=" * 80)

# Step 1: Chunk the documentation
print("\n📄 Chunking documentation...")
chunker = TextChunker()
chunks = chunker.chunk_by_paragraphs(
    aws_documentation,
    min_chunk_size=200,
    max_chunk_size=1000
)

print(f"✅ Created {len(chunks)} chunks")

# Step 2: Create Document objects
documents = []
for i, chunk in enumerate(chunks):
    # Extract service name from chunk
    first_line = chunk.split('\n')[0]
    
    doc = Document(
        content=chunk,
        metadata={
            'source': 'AWS Documentation',
            'chunk_id': i,
            'title': first_line[:50] if first_line else f'Chunk {i}'
        }
    )
    documents.append(doc)

# Step 3: Initialize vector store
print("\n🏗️ Initializing vector store...")
vector_store = RAGVectorStore()

# Step 4: Add documents
print("\n📊 Generating embeddings and building index...")
vector_store.add_documents(documents, batch_size=10)

# Step 5: Initialize RAG system
print("\n✅ Initializing RAG system...")
rag_system = RAGSystem(vector_store)

print("\n" + "="*80)
print("✅ RAG Knowledge Base Ready!")
print("="*80)

Building RAG Knowledge Base

📄 Chunking documentation...
✅ Created 4 chunks

🏗️ Initializing vector store...

📊 Generating embeddings and building index...
✅ Auto-detected dimension: 1536
✅ Added 4 documents

✅ Initializing RAG system...

✅ RAG Knowledge Base Ready!


## Test RAG System

In [21]:
# Test Query 1: Specific service question
query1 = "What is Amazon Bedrock and what capabilities does it provide?"

print("\n" + "#"*80)
print("TEST QUERY 1")
print("#"*80)

response1 = rag_system.query(query1, top_k=2)
rag_system.display_response(response1)


################################################################################
TEST QUERY 1
################################################################################
🔍 Retrieving top 2 relevant documents...
✅ Retrieved 2 documents
🤖 Generating answer...

QUESTION: What is Amazon Bedrock and what capabilities does it provide?

ANSWER:
According to [Document 1], Amazon Bedrock is a fully managed service that provides access to high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API. It offers capabilities to:

1. Easily experiment with and evaluate top foundation models for different use cases.
2. Privately customize the foundation models with your own data using techniques like fine-tuning and Retrieval Augmented Generation (RAG).
3. Build agents that can execute tasks using your enterprise systems and data sources.
4. Leverage security, privacy, and responsible AI features provided by

In [22]:
# Test Query 2: Comparison question
query2 = "What are the differences between Lambda and EC2?"

print("\n" + "#"*80)
print("TEST QUERY 2")
print("#"*80)

response2 = rag_system.query(query2, top_k=3)
rag_system.display_response(response2)


################################################################################
TEST QUERY 2
################################################################################
🔍 Retrieving top 3 relevant documents...
✅ Retrieved 3 documents
🤖 Generating answer...

QUESTION: What are the differences between Lambda and EC2?

ANSWER:
The key differences between AWS Lambda and Amazon EC2 based on the provided context are:

1. Compute model: Lambda is a serverless compute service where you run code without provisioning servers, while EC2 provides virtual servers that you manage and scale yourself [Document 1, Document 2].

2. Pricing: With Lambda, you pay only for the compute time used when your code runs. With EC2, you pay for the instances whether they are being utilized or not [Document 2].

3. Scalability: Lambda automatically scales the compute capacity based on incoming requests or events. With EC2, you need to manually scale instances up or down based on demand [Document 2].

4. Mana

In [23]:
# Test Query 3: Multi-hop reasoning
query3 = "How can I use SageMaker with S3 for machine learning?"

print("\n" + "#"*80)
print("TEST QUERY 3")
print("#"*80)

response3 = rag_system.query(query3, top_k=3)
rag_system.display_response(response3)


################################################################################
TEST QUERY 3
################################################################################
🔍 Retrieving top 3 relevant documents...
✅ Retrieved 3 documents
🤖 Generating answer...

QUESTION: How can I use SageMaker with S3 for machine learning?

ANSWER:
According to the provided context documents, you can use Amazon SageMaker with Amazon S3 for machine learning in the following ways:

1. SageMaker provides an integrated Jupyter notebook instance that allows easy access to your data sources, which can include data stored in Amazon S3, for exploration and analysis [Document 1].

2. SageMaker offers optimized machine learning algorithms that can efficiently run against extremely large datasets stored in Amazon S3 in a distributed environment [Document 1].

3. You can use SageMaker to quickly build, train, and deploy machine learning models, likely leveraging data stored in Amazon S3 [Document 2].

However,

In [24]:
# Test Query 4: Out-of-domain question
query4 = "How do I cook pasta?"

print("\n" + "#"*80)
print("TEST QUERY 4 (Out-of-Domain)")
print("#"*80)

response4 = rag_system.query(query4, top_k=2, min_score=0.5)
rag_system.display_response(response4)


################################################################################
TEST QUERY 4 (Out-of-Domain)
################################################################################
🔍 Retrieving top 2 relevant documents...

QUESTION: How do I cook pasta?

ANSWER:
I couldn't find relevant information to answer your question.

--------------------------------------------------------------------------------
SOURCES (0 documents):
--------------------------------------------------------------------------------



## RAG Evaluation Metrics

In [25]:
class RAGEvaluator:
    """Evaluate RAG system performance"""
    
    @staticmethod
    def calculate_retrieval_metrics(
        retrieved_docs: List[RetrievalResult],
        relevant_doc_ids: List[str]
    ) -> Dict[str, float]:
        """
        Calculate retrieval metrics
        
        Args:
            retrieved_docs: Retrieved documents
            relevant_doc_ids: Ground truth relevant document IDs
        
        Returns:
            Dictionary of metrics
        """
        retrieved_ids = {doc.document.doc_id for doc in retrieved_docs}
        relevant_set = set(relevant_doc_ids)
        
        # True positives
        tp = len(retrieved_ids & relevant_set)
        
        # Precision: TP / (TP + FP)
        precision = tp / len(retrieved_ids) if retrieved_ids else 0
        
        # Recall: TP / (TP + FN)
        recall = tp / len(relevant_set) if relevant_set else 0
        
        # F1 Score
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        # Mean Reciprocal Rank (MRR)
        mrr = 0
        for rank, doc in enumerate(retrieved_docs, 1):
            if doc.document.doc_id in relevant_set:
                mrr = 1 / rank
                break
        
        return {
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'mrr': mrr
        }
    
    @staticmethod
    def evaluate_answer_quality(
        response: RAGResponse,
        check_citation: bool = True
    ) -> Dict[str, any]:
        """
        Evaluate answer quality
        
        Args:
            response: RAG response
            check_citation: Whether to check for citations
        
        Returns:
            Quality metrics
        """
        answer = response.answer
        
        # Check for citations
        has_citations = bool(re.search(r'\[Document \d+\]', answer))
        
        # Check for uncertainty markers
        uncertainty_phrases = [
            "I don't know",
            "not enough information",
            "can't find",
            "unable to answer"
        ]
        has_uncertainty = any(phrase.lower() in answer.lower() for phrase in uncertainty_phrases)
        
        # Answer length
        answer_length = len(answer.split())
        
        # Number of sources used
        num_sources = len(response.sources)
        
        return {
            'has_citations': has_citations,
            'has_uncertainty': has_uncertainty,
            'answer_length_words': answer_length,
            'num_sources_retrieved': num_sources,
            'avg_source_score': np.mean([s.score for s in response.sources]) if response.sources else 0
        }

# Evaluate responses
print("\n" + "="*80)
print("RAG EVALUATION METRICS")
print("="*80)

evaluator = RAGEvaluator()

for i, response in enumerate([response1, response2, response3], 1):
    print(f"\nQuery {i}: {response.query[:60]}...")
    print("-" * 80)
    
    quality_metrics = evaluator.evaluate_answer_quality(response)
    
    for metric, value in quality_metrics.items():
        print(f"  {metric}: {value}")

print("\n" + "="*80)


RAG EVALUATION METRICS

Query 1: What is Amazon Bedrock and what capabilities does it provide...
--------------------------------------------------------------------------------
  has_citations: True
  has_uncertainty: False
  answer_length_words: 144
  num_sources_retrieved: 2
  avg_source_score: 0.5144682675600052

Query 2: What are the differences between Lambda and EC2?...
--------------------------------------------------------------------------------
  has_citations: True
  has_uncertainty: False
  answer_length_words: 180
  num_sources_retrieved: 3
  avg_source_score: 0.46221161882082623

Query 3: How can I use SageMaker with S3 for machine learning?...
--------------------------------------------------------------------------------
  has_citations: True
  has_uncertainty: False
  answer_length_words: 137
  num_sources_retrieved: 3
  avg_source_score: 0.4516541262467702



## Conversation History Analysis

In [26]:
# Analyze conversation history
print("\n" + "="*80)
print("CONVERSATION HISTORY SUMMARY")
print("="*80)

if rag_system.conversation_history:
    df_history = pd.DataFrame([
        {
            'Query': resp.query[:50] + '...',
            'Num Sources': len(resp.sources),
            'Avg Score': f"{np.mean([s.score for s in resp.sources]):.3f}" if resp.sources else 'N/A',
            'Answer Length': len(resp.answer.split()),
            'Timestamp': resp.timestamp[:19]
        }
        for resp in rag_system.conversation_history
    ])
    
    print(df_history.to_string(index=False))
else:
    print("No conversation history available.")


CONVERSATION HISTORY SUMMARY
                                                Query  Num Sources Avg Score  Answer Length           Timestamp
What is Amazon Bedrock and what capabilities does ...            2     0.514            144 2026-01-31T16:39:06
  What are the differences between Lambda and EC2?...            3     0.462            180 2026-01-31T16:39:16
How can I use SageMaker with S3 for machine learni...            3     0.452            137 2026-01-31T16:39:19


## Summary

In [27]:
print("""
═══════════════════════════════════════════════════════════════════════════════
                    RAG APPLICATION SUMMARY
═══════════════════════════════════════════════════════════════════════════════

✅ SUCCESSFULLY IMPLEMENTED:
   • Document chunking with multiple strategies
   • Vector store with FAISS for efficient retrieval
   • Cohere embeddings for semantic search
   • Claude 3 for contextual answer generation
   • Citation and source tracking
   • Conversation history management
   • RAG evaluation metrics

🎯 RAG PIPELINE:
   1. Document Ingestion → Chunking
   2. Embedding Generation → Vector Store
   3. Query → Semantic Retrieval
   4. Context Building → LLM Generation
   5. Response → Source Attribution

📊 KEY COMPONENTS:
   • TextChunker: Multiple chunking strategies
   • RAGVectorStore: FAISS-based vector search
   • RAGSystem: Complete pipeline orchestration
   • RAGEvaluator: Quality metrics

💡 BEST PRACTICES:
   1. Chunk documents at semantic boundaries (paragraphs/sections)
   2. Use appropriate chunk size (200-1000 characters)
   3. Include chunk overlap for context continuity
   4. Set minimum relevance threshold (0.3-0.5)
   5. Retrieve 3-5 documents for context
   6. Include source citations in prompts
   7. Monitor retrieval quality metrics
   8. Implement fallback for low-confidence answers

🔧 OPTIMIZATION OPPORTUNITIES:
   • Implement hybrid search (dense + sparse)
   • Add reranking layer
   • Cache frequent queries
   • Implement query expansion
   • Add multi-turn conversation support
   • Implement streaming responses

📈 PRODUCTION CONSIDERATIONS:
   • Scale vector store for large document sets
   • Implement incremental indexing
   • Add monitoring and logging
   • Implement rate limiting
   • Add error handling and retries
   • Consider cost optimization

═══════════════════════════════════════════════════════════════════════════════
""")


═══════════════════════════════════════════════════════════════════════════════
                    RAG APPLICATION SUMMARY
═══════════════════════════════════════════════════════════════════════════════

✅ SUCCESSFULLY IMPLEMENTED:
   • Document chunking with multiple strategies
   • Vector store with FAISS for efficient retrieval
   • Cohere embeddings for semantic search
   • Claude 3 for contextual answer generation
   • Citation and source tracking
   • Conversation history management
   • RAG evaluation metrics

🎯 RAG PIPELINE:
   1. Document Ingestion → Chunking
   2. Embedding Generation → Vector Store
   3. Query → Semantic Retrieval
   4. Context Building → LLM Generation
   5. Response → Source Attribution

📊 KEY COMPONENTS:
   • TextChunker: Multiple chunking strategies
   • RAGVectorStore: FAISS-based vector search
   • RAGSystem: Complete pipeline orchestration
   • RAGEvaluator: Quality metrics

💡 BEST PRACTICES:
   1. Chunk documents at semantic boundaries (paragraphs/